In [28]:
%pylab inline
import logging as log
log.getLogger().setLevel(log.DEBUG)
import pandas as pd
from ipypb import track
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
import regex
import psycopg2
from datetime import datetime

import glob

names = ["RADARGRAM COLUMN", "TIME", "LATITUDE", "LONGITUDE", "MARS RADIUS", "SPACECRAFT RADIUS", 
                 "RADIAL VELOCITY", "TANGENTIAL VELOCITY", "SZA", "PHASE/1.0E16"]

types = [int, datetime, float, float, float, float, float, float, float, float]
names = [n.replace(" ", "_").replace("/", "_").lower() for n in names]


dtype = [t for t in zip(names, types)]

geom_f = "/data/SciBigData/radargrams/geom/"
pattern = geom_f + "/**/*_geom.lbl"
log.info(pattern)

allgeoms = glob.glob(pattern)

log.info(f"found {len(allgeoms)} radargrams geom-files")

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['table', 'datetime', 'log', 'dtype']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
INFO:root:/data/SciBigData/radargrams/geom//**/*_geom.lbl
INFO:root:found 21871 radargrams geom-files


Populating the interactive namespace from numpy and matplotlib


In [29]:
import sqlite3

def lbl_to_geom(lbl_file):
    return lbl_file[:-3] + "tab"

def read_geom_lbl(file):
    o = np.genfromtxt(file, dtype=dtype, delimiter=",")
#     o = np.loadtxt(file, delimiter=",")
#     table = pd.DataFrame(o)
    
#     table = pd.read_csv(file, skiprows=0, names=names)
#     table = o[["latitude", "longitude"]]
    return o

def pandas_to_geopandas_points(df, xname="longitude", yname="latitude", crs=104971):
    geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
    crs = {'init': f'epsg:{crs}'} #http://www.spatialreference.org/ref/epsg/2263/
    geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)
    return geo_df

def match_id_from_geom_filename(fname):
    m = regex.match(".*_([0-9]*)_geom.lbl", fname)
    return np.int(m.group(1))

In [30]:
# c.close()
names = ["RADARGRAM COLUMN", "TIME", "LATITUDE", "LONGITUDE", "MARS RADIUS", "SPACECRAFT RADIUS", 
                 "RADIAL VELOCITY", "TANGENTIAL VELOCITY", "SZA", "PHASE/1.0E16"]
names = [n.replace(" ", "_").replace("/", "_").lower() for n in names]

types = [int, datetime, float, float, float, float, float, float, float, float]

dtype = [t for t in zip(names, types)]

In [31]:
con = psycopg2.connect(dbname='postgres', user='luca', host='localhost')
c = con.cursor()
# c.execute("BEGIN TRANSACTION;")
c.execute("COMMIT;")


c.execute("""SELECT
 pg_terminate_backend (pg_stat_activity.pid)
FROM
 pg_stat_activity
WHERE
 pg_stat_activity.datname = 'sharad';""")

c.execute("DROP DATABASE IF EXISTS sharad")
c.execute("CREATE DATABASE sharad")
# c.execute("CREATE EXTENSION postgis;")
# c.execute("CREATE EXTENSION postgis_topology;")

c.close()

In [32]:
# c = sqlite3.connect("mydb3.sqlite", isolation_level=None)
import psycopg2
conn = psycopg2.connect("dbname=sharad user=luca")
c = conn.cursor()


# c.execute('pragma journal_mode=wal')

c.execute("DROP TABLE IF EXISTS sharad;")


sql =  "CREATE TABLE sharad \
        (id SERIAL PRIMARY KEY , \
        longitude REAL NOT NULL, \
        latitude REAL NOT NULL, \
        radargram_column INTEGER NOT NULL,\
        track_id INTEGER NOT NULL\
        );"

c.execute(sql)



In [33]:
# sql = "SELECT AddGeometryColumn('sharad', 'geometry',\
#   104971, 'POINT', 'XY');"
# c.execute(sql)

In [34]:
out = {}
for fname in track(allgeoms):
    table = read_geom_lbl(lbl_to_geom(fname))
    table = np.column_stack([table["longitude"],table["latitude"], table["radargram_column"]])
    tid = match_id_from_geom_filename(fname)
#     c.execute("BEGIN TRANSACTION;")
    c.executemany(f"INSERT INTO sharad VALUES (DEFAULT,%s,%s,%s,{tid})", table)
#     c.execute("COMMIT;")

conn.commit()
c.close()

 [█████████████###############################################] 4801/21871 [30:50<00:01, 0.39s/it]

DiskFull: could not extend file "base/40366/40373": wrote only 4096 of 8192 bytes at block 43633
HINT:  Check free disk space.


In [104]:
import psycopg2
conn = psycopg2.connect("dbname=sharad user=luca")
c = conn.cursor()
c.execute("SELECT * FROM sharad")


UndefinedTable: relation "sharad" does not exist
LINE 1: SELECT * FROM sharad
                      ^


In [26]:
c.close()